In [1]:
import gym
from gym_power_trading.envs import PowerTradingEnv
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [3]:
file_name = 'data\AEP_PSGC1_AMP_long.h5'
path = Path(file_name).resolve()
uri = str(path)

In [2]:
df = pd.read_hdf(uri)
df.head()

NameError: name 'uri' is not defined

In [2]:
my_dataframe = pd.read_csv("RandomCloseValues.csv")

In [3]:
env = PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100))
obs = env.reset()
for _ in tqdm(range(100)):
    action = env.action_space.sample()
    obs, reward, done, truncated, info = env.step(action)
    print(info)
    if truncated:
        obs = env.reset()

100%|██████████| 100/100 [00:00<00:00, 24982.45it/s]

{'total_reward': 5.109017944335932, 'total_profit': 0.0, 'position': 1, 'battery_charge': 20}
{'total_reward': -372.0057281494139, 'total_profit': -375.2291723632811, 'position': 0, 'battery_charge': 0}
{'total_reward': -372.0057281494139, 'total_profit': -375.2291723632811, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -372.0057281494139, 'total_profit': -375.2291723632811, 'position': 0, 'battery_charge': 0.0}
{'total_reward': -371.03617843627916, 'total_profit': -375.2291723632811, 'position': 1, 'battery_charge': 20.0}
{'total_reward': -371.03617843627916, 'total_profit': -375.2291723632811, 'position': 2, 'battery_charge': 20.0}
{'total_reward': -371.03617843627916, 'total_profit': -375.2291723632811, 'position': 2, 'battery_charge': 20.0}
{'total_reward': -381.8406784057616, 'total_profit': -375.2291723632811, 'position': 1, 'battery_charge': 40.0}
{'total_reward': -447.18241119384754, 'total_profit': -440.2441964874266, 'position': 0, 'battery_charge': 20.0}
{'total_rew

In [3]:
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7

DEBUG:matplotlib:matplotlib data path: c:\Users\Andrew Deur\anaconda3\envs\rl_project\lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Andrew Deur\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\Andrew Deur\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\Andrew Deur\.matplotlib\fontlist-v330.json


In [4]:
venv = make_vec_env(lambda: PowerTradingEnv(df=my_dataframe, window_size=3, frame_bound=(10, 100)))

### Need to Modify pre-processing function, otherwise self._observation_space produces mismatch produces broadcast error during learning

In [6]:
model = DQN('MlpPolicy', venv)
model.learn(total_timesteps=10000,
            log_interval=1000,
            progress_bar=True)

Output()

DEBUG:Comm:handle_msg[0c5d9266fce143f2b9585641732c020a]({'header': {'date': datetime.datetime(2024, 4, 17, 23, 14, 29, 65000, tzinfo=tzutc()), 'msg_id': '4d79de46-9047-4694-8f13-0800491c4843', 'msg_type': 'comm_msg', 'session': '8079b571-7e49-4f21-9294-c5284bf6eb23', 'username': '264293a7-f70f-496f-9a7e-4703cf023644', 'version': '5.2'}, 'msg_id': '4d79de46-9047-4694-8f13-0800491c4843', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': '0c5d9266fce143f2b9585641732c020a', 'data': {'method': 'update', 'state': {'outputs': [{'output_type': 'display_data', 'data': {'text/plain': '\x1b  81%\x1b \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━\x1b \x1b8,076/10,000 \x1b [ \x1b0:00:00\x1b < \x1b0:00:01\x1b , \x1b9,994 it/s\x1b ]\n', 'text/html': '<pre style="white-space:pre;overflow-x:auto;line-height:normal;font-family:Menlo,\'DejaVu Sans Mono\',consolas,\'Courier New\',monospace"><span style="color: #800080; text-decoration-col